# BraTS2020 **2D** Segmentation

---
## 1. Loading data and preprocessing
### 1.1 Loading data and preprocessing
**Pipeline**
1. Group all `.h5` slices by patient (`volume_<vid>_slice_<sid>.h5`)
2. Patient-wise subset & train/val/test split (avoids leakage)
3. 2D preprocessing: drop all-zero masks, percentile normalization, center pad/crop

In [1]:
import os, re, random, h5py, numpy as np, pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from collections import defaultdict
from scipy.spatial.distance import cdist
import json

DATA_ROOT = Path('./brats2020-training-data/BraTS2020_training_data/content/data')
OUTPUT_DIR = Path('./outputs'); OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

SUBSET_FRACTION = 1.0 #0.02 # 1.0 # 0.5
SEED = 42
BATCH_SIZE = 192 # 96 # 64 # 128
EPOCHS = 60 # 80

ES_PATIENCE = 10
ES_MIN_DELTA = 1e-4

print(f'CPU count: {os.cpu_count()}')
num_workers = min(os.cpu_count(),  12)
print(f'Using {num_workers} data loader workers')
# num_workers = 0  # set >0 after debugging

LR = 1e-4
IMG_SIZE = (240, 240)
# DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
print(f'Using device: {DEVICE}')
print(f'Reading slices from: {DATA_ROOT.resolve()}')


CPU count: 8
Using 8 data loader workers
Using device: cuda
Reading slices from: /home/tanhuoyuan_gmail_com/DIT698_DML_Project_BraTS_Hightlight/brats2020-training-data/BraTS2020_training_data/content/data


In [2]:
H5_RE = re.compile(r'^volume_(\d+)_slice_(\d+)\.h5$', re.IGNORECASE)

def group_by_volume(root: Path):
    groups = defaultdict(list)
    for p in root.glob('*.h5'):
        m = H5_RE.fullmatch(p.name)
        if not m:
            continue
        vid = int(m.group(1)); sid = int(m.group(2))
        groups[vid].append((sid, p.resolve()))
    grouped = {vid: [path for sid, path in sorted(items, key=lambda x: x[0])]
               for vid, items in sorted(groups.items(), key=lambda kv: kv[0])}
    return grouped

groups = group_by_volume(DATA_ROOT)
print(f'Found {len(groups)} patients. Example IDs:', list(groups.keys())[:8])
total_slices = sum(len(v) for v in groups.values())
print(f'Total slices discovered: {total_slices}')


Found 369 patients. Example IDs: [1, 2, 3, 4, 5, 6, 7, 8]
Total slices discovered: 57195


In [3]:
all_vids = list(groups.keys())
random.Random(SEED).shuffle(all_vids)
subset_n = max(1, int(len(all_vids) * SUBSET_FRACTION))
subset_vids = sorted(all_vids[:subset_n])

train_vids, test_vids = train_test_split(subset_vids, test_size=0.2, random_state=SEED)
train_vids, val_vids  = train_test_split(train_vids, test_size=0.2, random_state=SEED)

def gather_paths(vid_list):
    out = []
    for vid in vid_list:
        out.extend(groups[vid])
    return out

train_paths = gather_paths(train_vids)
val_paths   = gather_paths(val_vids)
test_paths  = gather_paths(test_vids)

print(f'Subset patients: {len(subset_vids)} | train: {len(train_vids)}, val: {len(val_vids)}, test: {len(test_vids)}')
print(f'Slices -> train: {len(train_paths)}, val: {len(val_paths)}, test: {len(test_paths)}')


Subset patients: 369 | train: 236, val: 59, test: 74
Slices -> train: 36580, val: 9145, test: 11470


In [4]:
import h5py, numpy as np

def read_h5(path: Path):
    """
    Expect:
      image: (H, W, 4) or (4, H, W)  -> returns (4, H, W) float32
      mask:  (H, W, 3) one-hot for (Necrosis, Edema, Enhancing)
             -> returns (H, W) int64 with labels: 0=BG, 1=Necrosis, 2=Edema, 3=Enhancing
    """
    # with h5py.File(path, "r") as f:
    with h5py.File(path, "r", libver="latest", swmr=True) as f:
        img = f["image"][()]
        # to channels-first (4, H, W)
        if img.ndim == 3 and img.shape[-1] in (3,4):
            img = np.moveaxis(img, -1, 0)
        elif img.ndim == 2:
            img = img[None, ...]
        elif img.ndim == 3 and img.shape[0] in (3,4):
            pass
        else:
            raise ValueError(f"Unexpected image shape: {img.shape}")

        m = f["mask"][()]  # (H, W, 3) one-hot
        if m.ndim != 3 or m.shape[-1] != 3:
            raise ValueError(f"Expect mask (H,W,3) one-hot, got {m.shape}")

        # background channel = 1 - any(foreground)
        bg = (m <= 0).all(axis=-1, keepdims=True).astype(np.uint8)  # (H,W,1)
        # stack as [BG, Necrosis, Edema, Enhancing]
        four = np.concatenate([bg, (m > 0).astype(np.uint8)], axis=-1)  # (H,W,4)
        # to class indices 0..3
        cls = np.argmax(four, axis=-1).astype(np.int64)  # (H,W)

        return img.astype(np.float32), cls  # (4,H,W), (H,W) int64


def percentile_normalize(img, lower=0.5, upper=99.5, eps=1e-6):
    # img: (C,H,W)
    out = np.empty_like(img, dtype=np.float32)
    for c in range(img.shape[0]):
        lo = np.percentile(img[c], lower); hi = np.percentile(img[c], upper)
        out[c] = np.clip((img[c]-lo)/(hi-lo+eps), 0, 1)
    return out

def _pad2d_or_3d(arr, ph, pw):
    if arr.ndim == 3:  # (C,H,W)
        return np.pad(arr, ((0,0),(ph//2, ph-ph//2),(pw//2, pw-pw//2)), mode="constant")
    if arr.ndim == 2:  # (H,W)
        return np.pad(arr, ((ph//2, ph-ph//2),(pw//2, pw-pw//2)), mode="constant")
    raise ValueError(f"Unsupported ndim={arr.ndim}")

def center_pad_crop(img, mask, size=(240,240)):
    H, W = img.shape[1], img.shape[2]; th, tw = size
    ph, pw = max(0, th-H), max(0, tw-W)
    if ph>0 or pw>0:
        img  = _pad2d_or_3d(img,  ph, pw)
        mask = _pad2d_or_3d(mask, ph, pw)
    H2, W2 = img.shape[1], img.shape[2]
    sy, sx = (H2-th)//2, (W2-tw)//2
    return img[:, sy:sy+th, sx:sx+tw], mask[sy:sy+th, sx:sx+tw]


In [5]:
from torch.utils.data import Dataset, DataLoader

class BraTS2DDataset(Dataset):
    """
    Lazy-loading dataset:
      - Keep only file paths in __init__
      - Read/normalize/pad-crop on-the-fly in __getitem__
      - Optionally drop slices whose mask is all background
    """
    def __init__(self, paths, drop_all_zero=True, size=(240, 240)):
        self.paths = list(paths)
        self.drop_all_zero = drop_all_zero
        self.size = size

    def __len__(self):
        return len(self.paths)

    def _load_one(self, p: Path):
        # read from disk
        img, cls = read_h5(p)                 # img: (4,H,W)  cls: (H,W) int
        # optional drop: keep only slices with any foreground
        if self.drop_all_zero and cls.max() == 0:
            return None
        # preprocessing
        img = percentile_normalize(img)
        img, cls = center_pad_crop(img, cls, size=self.size)
        # to tensors
        x = torch.from_numpy(img).float()     # (C,H,W)
        y = torch.from_numpy(cls).long()      # (H,W)
        return x, y, p.name.replace('.h5','')

    def __getitem__(self, idx):
        p = self.paths[idx]
        sample = self._load_one(p)

        # If this slice was dropped due to all-zero mask, fallback to a nearby one
        if sample is None:
            # linear probing to find the next valid slice
            j = (idx + 1) % len(self.paths)
            while True:
                sample = self._load_one(self.paths[j])
                if sample is not None or j == idx:
                    break
                j = (j + 1) % len(self.paths)

            # as a last resort (all zero), create a dummy empty slice (rare)
            if sample is None:
                img, cls = read_h5(self.paths[idx])
                img = percentile_normalize(img)
                img, cls = center_pad_crop(img, cls, size=self.size)
                sample = (torch.from_numpy(img).float(),
                          torch.from_numpy(cls).long(),
                          self.paths[idx].name.replace('.h5',''))

        return sample


def make_loader_kwargs(num_workers: int):
    if num_workers == 0:
        return dict(num_workers=0, pin_memory=False)  # single-process, no prefetch/persistent
    else:
        return dict(
            num_workers=num_workers,
            pin_memory=True,
            persistent_workers=True,
            prefetch_factor=2,
        )

common_kwargs = make_loader_kwargs(num_workers)

train_ds = BraTS2DDataset(train_paths, drop_all_zero=True,  size=IMG_SIZE)
val_ds   = BraTS2DDataset(val_paths,   drop_all_zero=True,  size=IMG_SIZE)
test_ds  = BraTS2DDataset(test_paths,  drop_all_zero=False, size=IMG_SIZE)


train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  **common_kwargs)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, **common_kwargs)
test_loader  = DataLoader(test_ds,  batch_size=1,          shuffle=False, **common_kwargs)


---
### 1.2 Train/Evaluate Visualize

In [6]:
import numpy as np
import torch
import matplotlib.pyplot as plt

def viz_overlays(
    ds,
    model,
    device="cpu",
    n=6,
    rows=None,
    cols=None,
    base_channel=1,                 # which modality to display as grayscale (e.g., 1 = T1Gd)
    class_colors=None,              # dict like {1:'y', 2:'c', 3:'m'}
    gt_linestyle="solid",
    pred_linestyle="dashed",
    linewidth=1.2,
    figsize=(10, 6),
    seed=None,
):
    if class_colors is None:
        class_colors = {1: "y", 2: "c", 3: "m"}

    if seed is not None:
        np.random.seed(seed)

    if rows is None or cols is None:
        rows = rows or 2
        cols = cols or int(np.ceil(n / rows))

    model.eval()
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    axes = np.atleast_1d(axes).ravel()

    with torch.no_grad():
        for i in range(min(n, len(axes))):
            idx = np.random.randint(len(ds))
            x, y, name = ds[idx]                # x: (C,H,W), y: (H,W)
            xb = x[None].to(device)             # (1,C,H,W)

            logits = model(xb)                  # (1,4,H,W)
            pred = torch.argmax(torch.softmax(logits, dim=1), dim=1)[0].cpu().numpy()
            gt   = y.cpu().numpy() if torch.is_tensor(y) else y

            # choose background channel safely
            bch = base_channel
            if bch >= x.shape[0]:
                bch = 0
            base = x.numpy()[bch] if torch.is_tensor(x) else x[bch]

            ax = axes[i]
            ax.imshow(base, cmap="gray")

            # draw GT (solid) and Pred (dashed) for each foreground class
            for c in (1, 2, 3):
                ax.contour((gt==c).astype(np.uint8),   levels=[0.5],
                           colors=class_colors.get(c, "w"), linewidths=linewidth,
                           linestyles=gt_linestyle)
                ax.contour((pred==c).astype(np.uint8), levels=[0.5],
                           colors=class_colors.get(c, "w"), linewidths=linewidth,
                           linestyles=pred_linestyle)

            ax.set_title(str(name))
            ax.axis("off")

    plt.suptitle("Solid = GT  |  Dashed = Pred  (1: Necrosis, 2: Edema, 3: Enhancing)")
    plt.tight_layout()
    plt.show()

In [7]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

def viz_overlays_pro(
    ds,
    model,
    device="cpu",
    n=6,
    samples_per_row=3,            # how many cases per row (each case = 1 or 2 panels)
    base_channel=1,               # modality to display as grayscale
    show_error=False,             # also show TP/FP/FN
    panel_size=3.2,               # inches per panel (width ~= height)
    class_names={1:"Necrosis", 2:"Edema", 3:"Enhancing"},
    class_colors={1:"#ffcc00", 2:"#00c5d8", 3:"#cc66ff"},  # GT fill colors
    pred_edge_colors=None,        # None -> reuse fill colors
    gt_alpha=0.35,
    pred_lw=2.6,
    pred_ls="dashed",
    seed=None,
    suptitle="GT = filled  |  Pred = dashed outline",
):
    """
    Compact overlay visualization for multiclass segmentation.
      - 0 = background, 1..3 = foreground classes.
      - If `show_error=True`, each sample occupies two panels: [overlay | error].
    """
    if seed is not None:
        np.random.seed(seed)
    if pred_edge_colors is None:
        pred_edge_colors = {k: class_colors[k] for k in class_colors}

    model.eval()

    # how many panels per sample
    panels_per_sample = 2 if show_error else 1

    # grid sizes
    rows = int(np.ceil(n / samples_per_row))
    cols = samples_per_row * panels_per_sample

    # figure size (proportional to number of panels)
    fig_w = panel_size * cols
    fig_h = panel_size * rows
    fig, axes = plt.subplots(rows, cols, figsize=(fig_w, fig_h))
    axes = np.atleast_2d(axes)
    axes = axes.reshape(rows, cols)

    # helper: percentile normalization for the grayscale background
    def _norm(x, lo=1.0, hi=99.0, eps=1e-6):
        p1, p99 = np.percentile(x, lo), np.percentile(x, hi)
        x = (x - p1) / (p99 - p1 + eps)
        return np.clip(x, 0, 1)

    with torch.no_grad():
        for i in range(n):
            r = i // samples_per_row
            c0 = (i % samples_per_row) * panels_per_sample
            ax = axes[r, c0]

            # pick a random sample
            x, y, name = ds[np.random.randint(len(ds))]
            xb = x[None].to(device)
            logits = model(xb)
            pred = torch.argmax(logits, dim=1)[0].cpu().numpy()
            gt   = y.cpu().numpy() if torch.is_tensor(y) else y

            # choose background modality & normalize for better contrast
            bch  = base_channel if base_channel < x.shape[0] else 0
            base = x.numpy()[bch] if torch.is_tensor(x) else x[bch]
            base = _norm(base)

            # ---- Overlay panel
            ax.imshow(base, cmap="gray", interpolation="nearest")
            for cc in (1, 2, 3):
                gt_mask = (gt == cc).astype(np.uint8)
                if gt_mask.any():
                    ax.contourf(gt_mask, levels=[0.5, 1.5],
                                colors=[class_colors[cc]], alpha=gt_alpha, antialiased=False)
                pred_mask = (pred == cc).astype(np.uint8)
                if pred_mask.any():
                    ax.contour(pred_mask, levels=[0.5],
                               colors=[pred_edge_colors[cc]],
                               linewidths=pred_lw, linestyles=pred_ls, antialiased=False)
            ax.set_title(str(name), fontsize=10)
            ax.set_axis_off()
            ax.set_aspect("equal", adjustable="box")

            # ---- Error panel (optional)
            if show_error:
                ax2 = axes[r, c0 + 1]
                tp = (pred == gt) & (gt > 0)              # green
                fp = (pred > 0) & (gt == 0)               # red
                fn = (pred == 0) & (gt > 0)               # blue
                err = np.zeros((*gt.shape, 3), dtype=np.float32)
                err[tp] = (0.0, 0.9, 0.0)
                err[fp] = (1.0, 0.1, 0.1)
                err[fn] = (0.1, 0.6, 1.0)
                ax2.imshow(base, cmap="gray", interpolation="nearest")
                ax2.imshow(err, alpha=0.45, interpolation="nearest")
                ax2.set_title("Error (TP/FP/FN)", fontsize=10)
                ax2.set_axis_off()
                ax2.set_aspect("equal", adjustable="box")

    # tighten spacing; leave a small room for legend/suptitle
    plt.subplots_adjust(wspace=0.02, hspace=0.15)

    # single legend for the whole figure
    legend = []
    for c, nm in class_names.items():
        legend.append(Patch(facecolor=class_colors[c], edgecolor="none", alpha=gt_alpha, label=f"GT {nm}"))
        legend.append(Line2D([0],[0], color=pred_edge_colors[c], lw=pred_lw, linestyle=pred_ls, label=f"Pred {nm}"))
    if show_error:
        legend += [
            Patch(facecolor=(0,0.9,0), edgecolor='none', alpha=0.6, label="TP"),
            Patch(facecolor=(1,0.1,0.1), edgecolor='none', alpha=0.6, label="FP"),
            Patch(facecolor=(0.1,0.6,1), edgecolor='none', alpha=0.6, label="FN"),
        ]
    fig.legend(handles=legend, loc="upper center", ncol=5 if show_error else 3,
               bbox_to_anchor=(0.5, 1.02), frameon=False, fontsize=9)
    fig.suptitle(suptitle, y=1.06, fontsize=11)
    plt.tight_layout(rect=(0, 0, 1, 0.98))
    plt.show()


In [8]:
import numpy as np
import pandas as pd
import re
import torch

def dice_iou_binary(p, t, eps=1e-6):
    inter = (p & t).sum()
    union = (p | t).sum()
    dice = (2*inter + eps) / (p.sum() + t.sum() + eps)
    iou  = (inter + eps) / (union + eps)
    return float(dice), float(iou)

def evaluate_multiclass(
    loader,
    model,
    device="cpu",
    num_classes=4,
    add_overall_mean: bool = True,
    overall_label: str = "Overall_Mean",
    save_csv_dir = None,                  # e.g., OUTPUT_DIR Path; saves 2 CSVs if not None
    model_name=None
):
    """
    Evaluate a multiclass model on a DataLoader.

    Returns:
      per_slice_df : metrics per slice
      per_vol_df   : metrics aggregated per volume (with an extra 'Overall_Mean' row if add_overall_mean=True)
      overall_row  : dict of overall mean metrics (or None if add_overall_mean=False)
    """
    model.eval()
    rows = []
    with torch.no_grad():
        for x, y, name in loader:
            x = x.to(device)        # (1, C, H, W)
            y = y.to(device)        # (1, H, W) long
            logits = model(x)       # (1, num_classes, H, W)

            pred = torch.argmax(torch.softmax(logits, dim=1), dim=1)[0].cpu().numpy()
            gt   = y[0].cpu().numpy()

            metrics = {"id": name[0]}
            dices, ious = [], []
            for c, cname in zip([1, 2, 3], ["Necrosis", "Edema", "Enhancing"]):
                p = (pred == c).astype(np.uint8)
                t = (gt   == c).astype(np.uint8)
                d, i = dice_iou_binary(p, t)
                metrics[f"dice_{cname}"] = d
                metrics[f"iou_{cname}"]  = i
                dices.append(d); ious.append(i)

            metrics["dice_FG_mean"] = float(np.mean(dices))
            metrics["iou_FG_mean"]  = float(np.mean(ious))
            rows.append(metrics)

    per_slice_df = pd.DataFrame(rows)

    # ---- aggregate per volume
    def volid(s):
        m = re.match(r"(volume_\d+)_slice_\d+$", s)
        return m.group(1) if m else s

    per_slice_df["volume_id"] = per_slice_df["id"].map(volid)
    per_vol_df = per_slice_df.groupby("volume_id", as_index=False).mean(numeric_only=True)

    overall_row = None
    if add_overall_mean and len(per_vol_df) > 0:
        # compute overall mean across all numeric columns
        overall_row = per_vol_df.mean(numeric_only=True).to_dict()
        overall_row["volume_id"] = overall_label
        # append as the last row
        per_vol_df = pd.concat([per_vol_df, pd.DataFrame([overall_row])], ignore_index=True)

    # optional: save CSVs
    if save_csv_dir is not None:
        (save_csv_dir / f"{model_name}_metrics_mc_per_slice.csv").write_text(per_slice_df.to_csv(index=False))
        (save_csv_dir / f"{model_name}_metrics_mc_per_volume.csv").write_text(per_vol_df.to_csv(index=False))

    return per_slice_df, per_vol_df, overall_row

In [9]:
import numpy as np

def viz_compare(val_ds, model1, model2, device="cpu", n=6, model1_name=None, model2_name=None):
    """
    Visualize UNet vs Encoder–Decoder on the same slices.
    GT: solid contours, UNet: dashed, EncDec: dotted.
    """
    model1.eval(); model2.eval()
    rows, cols = (n + 2)//3, 3
    fig, axes = plt.subplots(rows, cols, figsize=(12, 4*rows))
    axes = axes.ravel()
    colors = {1:"y", 2:"c", 3:"m"}

    with torch.no_grad():
        for i in range(min(n, len(axes))):
            idx = np.random.randint(len(val_ds))
            x, y, name = val_ds[idx]
            base_img = x.numpy()[1] if x.shape[0] > 1 else x.numpy()[0]

            xb = x[None].to(device)
            pu = torch.argmax(torch.softmax(model1(xb),  dim=1), dim=1)[0].cpu().numpy()
            pe = torch.argmax(torch.softmax(model2(xb),    dim=1), dim=1)[0].cpu().numpy()
            gt = y.numpy()

            ax = axes[i]; ax.imshow(base_img, cmap="gray")
            for c in (1,2,3):
                ax.contour((gt==c).astype(np.uint8), levels=[0.5], colors=colors[c], linewidths=1.2)                 # GT (solid)
                ax.contour((pu==c).astype(np.uint8), levels=[0.5], colors=colors[c], linewidths=1.2, linestyles="dashed") # UNet
                ax.contour((pe==c).astype(np.uint8), levels=[0.5], colors=colors[c], linewidths=1.2, linestyles="dotted") # EncDec
            ax.set_title(f"{name} (solid=GT, dashed={model1_name}, dotted={model2_name})")
            ax.axis("off")
    plt.tight_layout(); plt.show()

In [10]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

def viz_compare_triplet(
    ds,
    model1, model2,
    device="cpu",
    n=6,
    samples_per_row=3,
    base_channel=1,
    class_names={1:"Necrosis", 2:"Edema", 3:"Enhancing"},
    # GT light fills
    gt_colors={1:"#ffe8a6", 2:"#bff6fb", 3:"#efd3ff"},
    gt_alpha=0.30,
    gt_edge=True,
    # UNet (model1) warm edges
    m1_colors={1:"#ffb000", 2:"#ff6f00", 3:"#a64cff"},
    # TL/EncDec (model2) cool edges
    m2_colors={1:"#007a99", 2:"#00a7b7", 3:"#5b6cff"},
    # line styles/widths
    m1_ls="--", m2_ls=":",
    m1_lw=3.0, m2_lw=3.0,
    # two-pass halo parameters
    m1_halo_color="black", m2_halo_color="white",
    m1_halo_extra=2.2, m2_halo_extra=2.2,
    panel_size=3.0,
    seed=None,
    title1="UNet", title2="TL"
):
    """
    GT = light filled polygons (+optional thin white edge).
    Model1 = dashed warm outline with black halo (two-pass).
    Model2 = dotted cool outline with white halo (two-pass).
    """
    if seed is not None:
        np.random.seed(seed)
    model1.eval(); model2.eval()

    rows = int(np.ceil(n / samples_per_row))
    cols = samples_per_row
    fig, axes = plt.subplots(rows, cols, figsize=(panel_size*cols, panel_size*rows))
    axes = np.atleast_1d(axes).ravel()

    def norm01(arr, lo=1, hi=99, eps=1e-6):
        p1, p99 = np.percentile(arr, lo), np.percentile(arr, hi)
        return np.clip((arr - p1) / (p99 - p1 + eps), 0, 1)

    with torch.no_grad():
        for i in range(min(n, len(axes))):
            idx = np.random.randint(len(ds))
            x, y, name = ds[idx]                   # x:(C,H,W), y:(H,W)
            xb = x[None].to(device)
            p1 = torch.argmax(model1(xb), dim=1)[0].cpu().numpy()
            p2 = torch.argmax(model2(xb), dim=1)[0].cpu().numpy()
            gt = y.cpu().numpy() if torch.is_tensor(y) else y

            ch   = base_channel if base_channel < x.shape[0] else 0
            base = x.numpy()[ch] if torch.is_tensor(x) else x[ch]
            base = norm01(base)

            ax = axes[i]
            ax.imshow(base, cmap="gray", interpolation="nearest")

            for c in (1,2,3):
                gtm = (gt==c).astype(np.uint8)
                if gtm.any():
                    ax.contourf(gtm, levels=[0.5,1.5],
                                colors=[gt_colors[c]], alpha=gt_alpha, antialiased=False, zorder=1)
                    if gt_edge:
                        ax.contour(gtm, levels=[0.5], colors=["#ffffff"],
                                   linewidths=0.8, linestyles="solid", antialiased=False, zorder=2)

                # --- model1 (UNet): HALO pass (behind), then COLOR pass (front)
                m1m = (p1==c).astype(np.uint8)
                if m1m.any():
                    ax.contour(m1m, levels=[0.5], colors=[m1_halo_color],
                               linewidths=m1_lw + m1_halo_extra, linestyles=m1_ls,
                               antialiased=False, zorder=4)   # halo
                    ax.contour(m1m, levels=[0.5], colors=[m1_colors[c]],
                               linewidths=m1_lw, linestyles=m1_ls,
                               antialiased=False, zorder=5)   # colored edge

                # --- model2 (TL): HALO pass (behind), then COLOR pass (front)
                m2m = (p2==c).astype(np.uint8)
                if m2m.any():
                    ax.contour(m2m, levels=[0.5], colors=[m2_halo_color],
                               linewidths=m2_lw + m2_halo_extra, linestyles=m2_ls,
                               antialiased=False, zorder=6)   # halo
                    ax.contour(m2m, levels=[0.5], colors=[m2_colors[c]],
                               linewidths=m2_lw, linestyles=m2_ls,
                               antialiased=False, zorder=7)   # colored edge

            ax.set_title(str(name), fontsize=12, pad=8)
            ax.set_axis_off(); ax.set_aspect("equal", adjustable="box")

    plt.subplots_adjust(wspace=0.03, hspace=0.18)

    # row A: classes via GT color
    palette = [Patch(facecolor=gt_colors[c], edgecolor="none", alpha=gt_alpha,
                    label=class_names[c]) for c in (1,2,3)]
    legA = fig.legend(
        handles=palette,                    # <--- use keyword
        title="Classes (GT fill)",
        loc="upper center",
        bbox_to_anchor=(0.5, 1.02),
        ncol=len(palette),
        frameon=False,
        fontsize=10,
        title_fontsize=10,
    )
    fig.add_artist(legA)

    # row B: style key
    style = [
        Line2D([0],[0], color="#333", lw=m1_lw+m1_halo_extra, ls=m1_ls, label=f"{title1} (dashed)"),
        Line2D([0],[0], color="#999", lw=m2_lw+m2_halo_extra, ls=m2_ls, label=f"{title2} (dotted)"),
        Patch(facecolor="#d9d9d9", edgecolor="none", alpha=0.6, label="GT = filled"),
    ]
    fig.legend(
        handles=style,                      # <--- use keyword
        loc="upper center",
        bbox_to_anchor=(0.5, 0.965),
        ncol=len(style),
        frameon=False,
        fontsize=10,
    )

    plt.tight_layout(rect=(0, 0, 1, 0.92))
    plt.show()



In [11]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

def _percentile_norm(img, lo=1.0, hi=99.0, eps=1e-6):
    p1, p99 = np.percentile(img, lo), np.percentile(img, hi)
    return np.clip((img - p1) / (p99 - p1 + eps), 0, 1)

def _binary_boundary(mask):
    """1-pixel boundary: mask XOR erode(mask). Works for uint8/0-1 arrays."""
    if mask.dtype != np.uint8:
        mask = mask.astype(np.uint8)
    # 4-neighborhood erosion
    from scipy.ndimage import binary_erosion
    er = binary_erosion(mask, structure=np.array([[0,1,0],[1,1,1],[0,1,0]], dtype=np.uint8), border_value=0)
    return (mask ^ er).astype(np.uint8)

def viz_compare_diagnostic(
    ds,
    model_1,
    model_2,
    device="cpu",
    n=6,
    samples_per_row=2,
    base_channel=1,
    show_overlay=True,
    show_boundaries=True,
    show_agreement=True,
    # colors
    gt_fill_colors   ={1:"#ffe9b0", 2:"#c7f5ff", 3:"#efd6ff"},   # light fills for overlay
    gt_edge_color    ="#ffffff",
    unet_edge_colors ={1:"#ff8a00", 2:"#ff4d00", 3:"#a64cff"},   # warm
    tl_edge_colors   ={1:"#008aa6", 2:"#00b1c4", 3:"#5b6cff"},   # cool
    # line styles
    unet_ls="--", tl_ls=":", unet_lw=2.8, tl_lw=2.8,
    gt_alpha=0.30, panel_size=3.1, seed=None,
    model_1_name=None, model_2_name=None
):
    """
    Triptych per slice:
      [Overlay]   GT = filled, UNet dashed (warm), TL dotted (cool)
      [Boundary]  Thin boundaries for GT(white), UNet(warm), TL(cool) -> shifts pop out
      [Agreement] Per-pixel fusion map that encodes who is positive for each class
    Assumes logits with 4 classes (0=BG, 1..3 FG) and y as (H,W) ints in [0..3].
    """
    if seed is not None: np.random.seed(seed)
    model_1.eval(); model_2.eval()

    # how many columns per sample
    cols_per_sample = int(show_overlay) + int(show_boundaries) + int(show_agreement)
    assert cols_per_sample >= 1, "Enable at least one of overlay/boundaries/agreement."

    rows = int(np.ceil(n / samples_per_row))
    cols = samples_per_row * cols_per_sample
    fig, axes = plt.subplots(rows, cols, figsize=(panel_size*cols, panel_size*rows))
    axes = np.atleast_2d(axes).reshape(rows, cols)

    # agreement color table (legend below)
    # label meaning (per pixel, per class):
    # 0: none; 1: GT only; 2: UNet only; 3: TL only; 4: UNet∩TL; 5: GT∩UNet; 6: GT∩TL; 7: ALL-3
    agree_colors = np.array([
        [0.00,0.00,0.00],  # 0 (unused background, not shown)
        [0.80,0.00,0.80],  # 1 GT only        -> magenta
        [1.00,0.55,0.00],  # 2 UNet only      -> orange
        [0.10,0.60,1.00],  # 3 TL only        -> blue
        [0.10,0.85,0.85],  # 4 UNet & TL      -> cyan
        [1.00,0.00,0.00],  # 5 GT & UNet      -> red
        [0.00,1.00,0.00],  # 6 GT & TL        -> green
        [1.00,1.00,0.00],  # 7 ALL-3          -> yellow
    ], dtype=np.float32)

    with torch.no_grad():
        for k in range(n):
            r = k // samples_per_row
            c0 = (k % samples_per_row) * cols_per_sample

            # sample & predictions
            x, y, name = ds[np.random.randint(len(ds))]
            xb = x[None].to(device)
            pu = torch.argmax(model_1(xb), dim=1)[0].cpu().numpy()
            pt = torch.argmax(model_2(xb),   dim=1)[0].cpu().numpy()
            gt = y.cpu().numpy() if torch.is_tensor(y) else y

            # background
            ch   = base_channel if base_channel < x.shape[0] else 0
            base = x.numpy()[ch] if torch.is_tensor(x) else x[ch]
            base = _percentile_norm(base)

            col_cursor = c0

            # ---------- Panel A: Overlay ----------
            if show_overlay:
                ax = axes[r, col_cursor]; col_cursor += 1
                ax.imshow(base, cmap="gray", interpolation="nearest")
                for c in (1,2,3):
                    gtm = (gt==c).astype(np.uint8)
                    if gtm.any():
                        ax.contourf(gtm, levels=[0.5,1.5], colors=[gt_fill_colors[c]],
                                    alpha=gt_alpha, antialiased=False, zorder=1)
                        ax.contour(gtm, levels=[0.5], colors=[gt_edge_color],
                                   linewidths=0.8, linestyles="solid", antialiased=False, zorder=2)
                    um = (pu==c).astype(np.uint8)
                    if um.any():
                        ax.contour(um, levels=[0.5], colors=[unet_edge_colors[c]],
                                   linewidths=unet_lw, linestyles=unet_ls, antialiased=False, zorder=4)
                    tm = (pt==c).astype(np.uint8)
                    if tm.any():
                        ax.contour(tm, levels=[0.5], colors=[tl_edge_colors[c]],
                                   linewidths=tl_lw, linestyles=tl_ls, antialiased=False, zorder=5)
                ax.set_title(str(name), fontsize=12, pad=8)
                ax.set_axis_off(); ax.set_aspect("equal", adjustable="box")

            # ---------- Panel B: Boundaries ----------
            if show_boundaries:
                ax = axes[r, col_cursor]; col_cursor += 1
                ax.imshow(base, cmap="gray", interpolation="nearest")
                for c in (1,2,3):
                    gb = _binary_boundary(gt==c)
                    ub = _binary_boundary(pu==c)
                    tb = _binary_boundary(pt==c)
                    if gb.any(): ax.imshow(np.ma.masked_where(gb==0, gb), cmap="gray", vmin=0, vmax=1,
                                           alpha=0.9, interpolation="nearest")  # white (see note below)
                    if ub.any(): ax.contour(ub, levels=[0.5], colors=[unet_edge_colors[c]],
                                            linewidths=unet_lw+1.6, linestyles=unet_ls, zorder=4)
                    if tb.any(): ax.contour(tb, levels=[0.5], colors=[tl_edge_colors[c]],
                                            linewidths=tl_lw+1.6, linestyles=tl_ls, zorder=5)
                # white GT boundary via imshow above (fast, bold); if you prefer contour:
                # ax.contour(gb, levels=[0.5], colors=[gt_edge_color], linewidths=2.0)
                ax.set_title("Boundaries", fontsize=11)
                ax.set_axis_off(); ax.set_aspect("equal", adjustable="box")

            # ---------- Panel C: Agreement / Disagreement ----------
            if show_agreement:
                ax = axes[r, col_cursor]; col_cursor += 1
                ax.imshow(base, cmap="gray", interpolation="nearest")
                H,W = gt.shape
                rgb = np.zeros((H,W,3), np.float32)
                for c in (1,2,3):
                    g = (gt==c); u=(pu==c); t=(pt==c)
                    code = (g.astype(np.uint8)<<2) | (u.astype(np.uint8)<<1) | t.astype(np.uint8)
                    # map non-zero codes to colors; zero stays background (transparent)
                    visible = code>0
                    rgb[visible] = agree_colors[code[visible]]
                ax.imshow(rgb, alpha=0.60, interpolation="nearest")
                ax.set_title("Agreement map", fontsize=11)
                ax.set_axis_off(); ax.set_aspect("equal", adjustable="box")

    # layout & legends
    plt.subplots_adjust(wspace=0.04, hspace=0.15)

    # Legend 1: classes (GT fill)
    class_handles = [Patch(facecolor=gt_fill_colors[c], edgecolor="none", alpha=gt_alpha,
                           label={1:"Necrosis",2:"Edema",3:"Enhancing"}[c]) for c in (1,2,3)]
    legA = fig.legend(handles=class_handles, title="Classes (GT fill)",
                      loc="upper center", bbox_to_anchor=(0.5, 1.02),
                      ncol=3, frameon=False, fontsize=10, title_fontsize=10)
    fig.add_artist(legA)

    # Legend 2: styles for models
    style_handles = [
        Line2D([0],[0], color="#444", lw=unet_lw, ls=unet_ls, label=f"{model_1_name} (dashed)"),
        Line2D([0],[0], color="#888", lw=tl_lw,   ls=tl_ls,   label=f"{model_2_name} (dotted)"),
        Patch(facecolor="#d9d9d9", edgecolor="none", alpha=0.6, label="GT = filled"),
    ]
    fig.legend(handles=style_handles, loc="upper center", bbox_to_anchor=(0.5, 0.965),
               ncol=3, frameon=False, fontsize=10)

    # Agreement legend
    agree_handles = [
        Patch(color=agree_colors[1], label="GT only"),
        Patch(color=agree_colors[2], label=f"{model_1_name} only"),
        Patch(color=agree_colors[3], label=f"{model_2_name} only"),
        Patch(color=agree_colors[4], label=f"{model_1_name} & {model_2_name}"),
        Patch(color=agree_colors[5], label=f"GT & {model_1_name}"),
        Patch(color=agree_colors[6], label=f"GT & {model_2_name}"),
        Patch(color=agree_colors[7], label="ALL-3"),
    ]
    fig.legend(handles=agree_handles, loc="lower center", bbox_to_anchor=(0.5, -0.02),
               ncol=4, frameon=False, fontsize=9)

    plt.tight_layout(rect=(0, 0.03, 1, 0.94))
    plt.show()


---
### 1.3 Loss Function
Combined Cross-Entropy and Multi-class SoftDice Loss:

In [12]:
# Combined Cross-Entropy + Multi-class SoftDice Loss
ce = nn.CrossEntropyLoss()  # logits: (B,4,H,W); target: (B,H,W) long

def soft_dice_mc(logits, target, eps=1e-6):
    """
    Multiclass soft Dice averaged over foreground classes (1..3).
    logits: (B,C,H,W), target: (B,H,W) long in [0..C-1]
    """
    probs = torch.softmax(logits, dim=1)               # (B,C,H,W)
    onehot = torch.zeros_like(probs)                   # (B,C,H,W)
    onehot.scatter_(1, target.unsqueeze(1), 1)         # GT one-hot

    # foreground only
    p = probs[:, 1:, :, :]                             # (B,3,H,W),foreground only
    t = onehot[:, 1:, :, :]
    inter = (p * t).sum(dim=(2,3)) * 2                 # (B,3)
    den   = (p*p + t*t).sum(dim=(2,3)) + eps           # (B,3)
    dice_per_class = inter / den                       # (B,3)
    return 1.0 - dice_per_class.mean()                 # scalar loss

def criterion(logits, target):
    return 0.5 * ce(logits, target) + 0.5 * soft_dice_mc(logits, target)

In [13]:
import gc, torch

def cleanup_cuda(model=None, optimizer=None, scheduler=None, scaler=None):
    if model is not None:
        try: model.to("cpu")
        except: pass
    del model, optimizer, scheduler, scaler
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()


---
### 1.5 Train Loop


In [14]:
import json, torch, numpy as np
from pathlib import Path

# ---------- metric: foreground macro-Dice (classes 1..C-1) ----------
@torch.no_grad()
def fg_macro_dice_from_logits(logits, y, num_classes=4, eps=1e-6):
    pred = torch.argmax(logits, dim=1)          # (B,H,W)
    dices = []
    for c in range(1, num_classes):
        p = (pred == c).float()
        t = (y    == c).float()
        inter = (p * t).sum(dim=(1,2)) * 2
        den   = (p * p + t * t).sum(dim=(1,2)) + eps
        dices.append(inter / den)
    return torch.stack(dices, 0).mean().item()

# ---------- early stopping ----------
class EarlyStopping:
    def __init__(self, mode='min', patience=5, min_delta=0.0, ckpt_path=None, verbose=True):
        assert mode in ('min', 'max')
        self.mode, self.patience, self.min_delta = mode, patience, min_delta
        self.verbose, self.ckpt_path = verbose, ckpt_path
        self.best = float('inf') if mode == 'min' else -float('inf')
        self.num_bad, self.should_stop = 0, False
        self.is_better = (lambda cur, best: (best - cur) > min_delta) if mode=='min' \
                         else (lambda cur, best: (cur - best) > min_delta)

    def step(self, value, model=None):
        if self.is_better(value, self.best):
            self.best, self.num_bad = value, 0
            if self.verbose:
                print(f"[EarlyStopping] best {self.mode}: {self.best:.6f}")
            if model is not None and self.ckpt_path is not None:
                torch.save(model.state_dict(), self.ckpt_path)
        else:
            self.num_bad += 1
            if self.verbose:
                print(f"[EarlyStopping] no improvement ({self.num_bad}/{self.patience})")
            if self.num_bad >= self.patience:
                self.should_stop = True
        return self.should_stop

# ---------- one epoch train ----------
def train_one_epoch(
    model, loader, optimizer, criterion, device, scaler,
    scheduler=None, scheduler_step='batch',    # 'batch' | 'epoch' | 'plateau'
    use_amp=True
):
    model.train()
    total, n = 0.0, 0
    optimizer.zero_grad(set_to_none=True)
    for x, y, _ in loader:
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)
        with torch.cuda.amp.autocast(enabled=use_amp):
            logits = model(x)
            loss = criterion(logits, y)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad(set_to_none=True)

        if scheduler is not None and scheduler_step == 'batch':
            scheduler.step()

        total += loss.item() * x.size(0)
        n += x.size(0)

    if scheduler is not None and scheduler_step == 'epoch':
        scheduler.step()

    return total / max(1, n)

# ---------- validation ----------
@torch.no_grad()
def validate(model, loader, criterion, device, num_classes=4, use_amp=False):
    model.eval()
    total_loss, total_dice, n = 0.0, 0.0, 0
    for x, y, _ in loader:
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)
        with torch.cuda.amp.autocast(enabled=use_amp):
            logits = model(x)
            loss = criterion(logits, y)
        total_loss += loss.item() * x.size(0)
        total_dice += fg_macro_dice_from_logits(logits, y, num_classes=num_classes) * x.size(0)
        n += x.size(0)
    return total_loss / max(1, n), total_dice / max(1, n)

# ---------- high-level fit ----------
def fit(
    model, train_loader, val_loader,
    criterion, optimizer, device,
    epochs,
    scheduler=None, scheduler_step='batch',      # 'batch' | 'epoch' | 'plateau'
    use_amp=None,
    early_stopping=None,                         # instance of EarlyStopping or None
    ckpt_path=None, history_path=None, num_classes=4, model_name="Model"
):
    if use_amp is None:
        use_amp = (device.type == "cuda")
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

    history = {"epoch": [], "train_loss": [], "val_loss": [], "val_dice": []}

    for ep in range(1, epochs + 1):
        tr = train_one_epoch(
            model, train_loader, optimizer, criterion, device, scaler,
            scheduler=scheduler, scheduler_step=scheduler_step, use_amp=use_amp
        )
        va, vd = validate(model, val_loader, criterion, device,
                          num_classes=num_classes, use_amp=False)

        # ReduceLROnPlateau: step on val metric
        if scheduler is not None and scheduler_step == 'plateau':
            # monitor val_loss
            scheduler.step(va)

        history["epoch"].append(ep)
        history["train_loss"].append(tr)
        history["val_loss"].append(va)
        history["val_dice"].append(vd)

        print(f"[{model_name} {ep:03d}] train={tr:.4f}  val={va:.4f}  dice={vd:.4f}")

        # early stopping, save best model
        if early_stopping is not None:
            stop = early_stopping.step(va, model=model)   # Monitor val_loss; if want val_dice, change here
            if stop:
                print(f"[EarlyStopping] stop at epoch {ep}")
                break

        # Save history after each epoch
        if history_path is not None:
            with open(history_path, "w") as f:
                json.dump(history, f)

    # Then load best checkpoint
    if early_stopping is not None and early_stopping.ckpt_path is not None:
        p = Path(early_stopping.ckpt_path)
        if p.exists():
            model.load_state_dict(torch.load(p, map_location=device))
            print(f"Loaded best checkpoint from {p}")

    if history_path is not None:
        with open(history_path, "w") as f:
            json.dump(history, f)

    return history


---
## 2. Train 2D Encode-Decode model

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import json

class ConvBNReLU(nn.Module):
    """(Conv -> BN -> ReLU) with same padding."""
    def __init__(self, in_ch, out_ch, k=3):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, k, padding=k//2, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )
    def forward(self, x): return self.block(x)

class EncoderDecoder2D(nn.Module):
    """
    Plain encoder–decoder:
      Encoder: repeated Conv blocks + MaxPool (downsampling)
      Bottleneck: Conv blocks
      Decoder: bilinear upsampling + Conv blocks (NO skip connections)
    """
    def __init__(self, in_ch=4, out_ch=4, base=32):
        super().__init__()
        # Encoder
        self.e1 = nn.Sequential(ConvBNReLU(in_ch, base),    ConvBNReLU(base, base))
        self.p1 = nn.MaxPool2d(2)
        self.e2 = nn.Sequential(ConvBNReLU(base, base*2),    ConvBNReLU(base*2, base*2))
        self.p2 = nn.MaxPool2d(2)
        self.e3 = nn.Sequential(ConvBNReLU(base*2, base*4),  ConvBNReLU(base*4, base*4))
        self.p3 = nn.MaxPool2d(2)
        self.e4 = nn.Sequential(ConvBNReLU(base*4, base*8),  ConvBNReLU(base*8, base*8))
        self.p4 = nn.MaxPool2d(2)

        # Bottleneck
        self.bott = nn.Sequential(ConvBNReLU(base*8, base*16), ConvBNReLU(base*16, base*16))

        # Decoder (no skip connections)
        self.d4 = nn.Sequential(ConvBNReLU(base*16, base*8),  ConvBNReLU(base*8, base*8))
        self.d3 = nn.Sequential(ConvBNReLU(base*8,  base*4),  ConvBNReLU(base*4, base*4))
        self.d2 = nn.Sequential(ConvBNReLU(base*4,  base*2),  ConvBNReLU(base*2, base*2))
        self.d1 = nn.Sequential(ConvBNReLU(base*2,  base),    ConvBNReLU(base,   base))

        self.outc = nn.Conv2d(base, out_ch, kernel_size=1)

    def up(self, x, scale=2):  # bilinear upsample
        return F.interpolate(x, scale_factor=scale, mode="bilinear", align_corners=False)

    def forward(self, x):
        # Encode
        x1 = self.e1(x); x = self.p1(x1)
        x2 = self.e2(x); x = self.p2(x2)
        x3 = self.e3(x); x = self.p3(x3)
        x4 = self.e4(x); x = self.p4(x4)

        # Bottleneck
        x  = self.bott(x)

        # Decode (no concatenations)
        x = self.up(x);  x = self.d4(x)
        x = self.up(x);  x = self.d3(x)
        x = self.up(x);  x = self.d2(x)
        x = self.up(x);  x = self.d1(x)

        return self.outc(x)  # logits (B, C, H, W)


In [16]:
in_channels = 4
num_classes = 4

encdec = EncoderDecoder2D(in_ch=in_channels, out_ch=num_classes).to(DEVICE)
print(f"Encoder-Decoder params: {sum(p.numel() for p in encdec.parameters())/1e6:.2f} M")


LR_ENCDEC = LR
optimizer_ed = torch.optim.Adam(encdec.parameters(), lr=LR_ENCDEC)

steps_per_epoch = len(train_loader)
scheduler_ed = torch.optim.lr_scheduler.OneCycleLR(
    optimizer_ed,
    max_lr=LR_ENCDEC,            # max learning rate
    steps_per_epoch=steps_per_epoch,
    epochs=EPOCHS,
    pct_start=0.1,               #  10% of cycle is warm-up
    div_factor=1e3,              # initial LR = max_lr / div_factor
    final_div_factor=1e4,        # min LR = initial LR / final_div_factor
    anneal_strategy='cos',       # cosine annealing
)

Encoder-Decoder params: 7.07 M


In [17]:
use_amp = (DEVICE == "cuda")
scaler_ed = torch.cuda.amp.GradScaler(enabled=use_amp)
es_ed = EarlyStopping(mode='min', patience=ES_PATIENCE, min_delta=ES_MIN_DELTA,
                   ckpt_path=OUTPUT_DIR/'encdec_best.pt')

ed_history = fit(
    model=encdec,
    train_loader=train_loader, val_loader=val_loader,
    criterion=criterion, optimizer=optimizer_ed, device=DEVICE, epochs=EPOCHS,
    scheduler=scheduler_ed, scheduler_step='batch',
    use_amp=(DEVICE.type=='cuda'),
    early_stopping=es_ed,
    ckpt_path=OUTPUT_DIR/'encdec_best.pt',
    history_path=OUTPUT_DIR/'encdec_training_history.json',
    model_name="EncDec", num_classes=4
)

# # load ed_history from file
# with open(OUTPUT_DIR/'encdec_training_history.json', 'r') as f:
#     ed_history = json.load(f)


/home/tanhuoyuan_gmail_com/conda/envs/dml/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:84.)
  return F.conv2d(input, weight, bias, self.stride,


[EncDec 001] train=1.1290  val=0.9962  dice=0.0365
[EarlyStopping] best min: 0.996158
[EncDec 002] train=0.9222  val=0.9168  dice=0.0578
[EarlyStopping] best min: 0.916815
[EncDec 003] train=0.8479  val=0.9564  dice=0.0799
[EarlyStopping] no improvement (1/10)
[EncDec 004] train=0.7850  val=0.7447  dice=0.1321
[EarlyStopping] best min: 0.744729
[EncDec 005] train=0.7038  val=0.6662  dice=0.1564
[EarlyStopping] best min: 0.666156
[EncDec 006] train=0.6242  val=0.5979  dice=0.1682
[EarlyStopping] best min: 0.597857


KeyboardInterrupt: 

In [ ]:
# with open(OUTPUT_DIR/'encdec_training_history.json', 'w') as f:
#     json.dump(ed_history, f)

plt.figure(figsize=(6,4)); plt.plot(ed_history['epoch'], ed_history['train_loss'], label='train')
plt.plot(ed_history['epoch'], ed_history['val_loss'], label='val'); plt.xlabel('Epoch'); plt.ylabel('Loss')
plt.title('Loss Curves'); plt.legend(); plt.tight_layout(); plt.show()

plt.figure(figsize=(6,4)); plt.plot(ed_history['epoch'], ed_history['val_dice'], label='val_dice')
plt.xlabel('Epoch'); plt.ylabel('Dice'); plt.title('Validation Dice'); plt.legend(); plt.tight_layout(); plt.show()

In [ ]:
encdec.load_state_dict(torch.load(OUTPUT_DIR/'encdec_best.pt', map_location=DEVICE))
encdec.eval()
# viz_overlays(val_ds, encdec, device=DEVICE, n=6, base_channel=1, seed=35)
viz_overlays_pro(val_ds, encdec, device=DEVICE, n=6, samples_per_row=2, show_error=True, base_channel=1, seed=35)

In [ ]:
ed_slice,   ed_vol,   ed_overall = evaluate_multiclass(
    test_loader, encdec, device=DEVICE, add_overall_mean=True, save_csv_dir=OUTPUT_DIR, model_name="encdec"
)
print("Overall mean row:")
print(ed_overall)          # {'dice_Necrosis':..., 'dice_Edema':..., ..., 'volume_id':'Overall_Mean'}
display(ed_vol.tail(3))  # Overall_Mean

In [ ]:
cleanup_cuda(model=encdec, optimizer=optimizer_ed, scheduler=scheduler_ed, scaler=scaler_ed)

---
## 3. Train 2D U-Net
### 3.1 Train 2D U-Net

1. Train a small 2D U-Net (4-channel input → 1-channel tumor mask)
2. Plot training curves & qualitative overlays
3. Evaluate on test set: Dice / IoU / 2D-HD95 (per-slice) and aggregate per-volume

In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )
    def forward(self, x):
        return self.net(x)

class UNet2D(nn.Module):
    def __init__(self, in_ch=4, out_ch=1, base=32):
        super().__init__()
        self.d1 = DoubleConv(in_ch, base);  self.p1 = nn.MaxPool2d(2)
        self.d2 = DoubleConv(base, base*2); self.p2 = nn.MaxPool2d(2)
        self.d3 = DoubleConv(base*2, base*4); self.p3 = nn.MaxPool2d(2)
        self.d4 = DoubleConv(base*4, base*8)
        self.u3 = nn.ConvTranspose2d(base*8, base*4, 2, stride=2); self.ud3 = DoubleConv(base*8, base*4)
        self.u2 = nn.ConvTranspose2d(base*4, base*2, 2, stride=2); self.ud2 = DoubleConv(base*4, base*2)
        self.u1 = nn.ConvTranspose2d(base*2, base, 2, stride=2);   self.ud1 = DoubleConv(base*2, base)
        self.outc = nn.Conv2d(base, out_ch, 1)
    def forward(self, x):
        x1 = self.d1(x)
        x2 = self.d2(self.p1(x1))
        x3 = self.d3(self.p2(x2))
        x4 = self.d4(self.p3(x3))
        x = self.u3(x4); x = self.ud3(torch.cat([x, x3], dim=1))
        x = self.u2(x);  x = self.ud2(torch.cat([x, x2], dim=1))
        x = self.u1(x);  x = self.ud1(torch.cat([x, x1], dim=1))
        return self.outc(x)

model = UNet2D(in_ch=4, out_ch=4).to(DEVICE)
print(f'Model params: {sum(p.numel() for p in model.parameters())/1e6:.2f} M')


In [ ]:
import json

history = {'epoch':[], 'train_loss':[], 'val_loss':[], 'val_dice':[]}
use_amp = (DEVICE == "cuda")
scaler_unet = torch.cuda.amp.GradScaler(enabled=use_amp)

optimizer_unet = torch.optim.Adam(model.parameters(), lr=LR)
steps_per_epoch = len(train_loader)
scheduler_unet = torch.optim.lr_scheduler.OneCycleLR(
    optimizer_unet,
    max_lr=LR,            # max learning rate
    steps_per_epoch=steps_per_epoch,
    epochs=EPOCHS,
    pct_start=0.1,               
    anneal_strategy='cos',
)


es_unet = EarlyStopping(mode='min', patience=ES_PATIENCE, min_delta=ES_MIN_DELTA,
                   ckpt_path=OUTPUT_DIR/'unet_best.pt')

hist_unet = fit(
    model=model,
    train_loader=train_loader, val_loader=val_loader,
    criterion=criterion, optimizer=optimizer_unet, device=DEVICE, epochs=EPOCHS,
    scheduler=scheduler_unet, scheduler_step='batch',
    use_amp=(DEVICE.type=='cuda'),
    early_stopping=es_unet,
    ckpt_path=OUTPUT_DIR/'unet_best.pt',
    history_path=OUTPUT_DIR/'unet2d_training_history.json',
    model_name="UNet", num_classes=4
)



In [ ]:
# # load history from file
# with open(OUTPUT_DIR/'unet2d_training_history.json', 'r') as f:
#     hist_unet = json.load(f)

plt.figure(figsize=(6,4)); plt.plot(hist_unet['epoch'], hist_unet['train_loss'], label='train')
plt.plot(hist_unet['epoch'], hist_unet['val_loss'], label='val'); plt.xlabel('Epoch'); plt.ylabel('Loss')
plt.title('Loss Curves'); plt.legend(); plt.tight_layout(); plt.show()

plt.figure(figsize=(6,4)); plt.plot(hist_unet['epoch'], hist_unet['val_dice'], label='val_dice')
plt.xlabel('Epoch'); plt.ylabel('Dice'); plt.title('Validation Dice'); plt.legend(); plt.tight_layout(); plt.show()




In [ ]:
# model.load_state_dict(torch.load(OUTPUT_DIR/'unet2d_best.pt', map_location=DEVICE))
model.eval()
# viz_overlays(val_ds, model, device=DEVICE, n=6, base_channel=1, seed=35)
viz_overlays_pro(val_ds, model, device=DEVICE, n=6, samples_per_row=2, show_error=True, base_channel=1, seed=35)


In [ ]:
unet_slice, unet_vol, unet_overall = evaluate_multiclass(
    test_loader, model, device=DEVICE, add_overall_mean=True, save_csv_dir=OUTPUT_DIR, model_name="unet"
)

print("Overall mean row:")
print(unet_overall)          # {'dice_Necrosis':..., 'dice_Edema':..., ..., 'volume_id':'Overall_Mean'}
display(unet_vol.tail(3))  # Overall_Mean


---
### 3.2 Compare with Endode-Decode Model

In [ ]:
import matplotlib.pyplot as plt
import json

# with open(OUTPUT_DIR/'unet2d_training_history.json', 'r') as f:
#     hist_unet = json.load(f)
with open(OUTPUT_DIR/'encdec_training_history.json', 'r') as f:
    ed_history = json.load(f)

plt.figure(figsize=(6,4))

plt.plot(hist_unet["epoch"], hist_unet["val_loss"], label="UNet val loss")
plt.plot(ed_history["epoch"], ed_history["val_loss"], label="EncDec val loss")
plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title("Validation Loss"); plt.legend(); plt.tight_layout(); plt.show()

plt.plot(hist_unet["epoch"], hist_unet["val_dice"], label="UNet val dice")
plt.plot(ed_history["epoch"], ed_history["val_dice"], label="EncDec val dice")
plt.xlabel("Epoch"); plt.ylabel("Foreground Dice"); plt.title("Validation Dice"); plt.legend(); plt.tight_layout(); plt.show()


In [ ]:
# # Load UNet
# model = UNet2D(in_ch=in_channels, out_ch=num_classes).to(DEVICE)
# model.load_state_dict(torch.load(OUTPUT_DIR/"unet2d_best.pt", map_location=DEVICE))
# model.eval()

# Load best Encoder–Decoder
encdec = EncoderDecoder2D(in_ch=in_channels, out_ch=num_classes).to(DEVICE)
encdec.load_state_dict(torch.load(OUTPUT_DIR/"encdec_best.pt", map_location=DEVICE))
encdec.eval()

# # Evaluate
# unet_slice, unet_vol, unet_overall = evaluate_multiclass(test_loader, model, device=DEVICE, model_name="unet")
ed_slice,   ed_vol,   ed_overall   = evaluate_multiclass(test_loader, encdec,   device=DEVICE, model_name="encdec")

# Merge per-volume tables and append overall mean row
import pandas as pd
cmp = unet_vol.merge(ed_vol, on="volume_id", suffixes=("_unet","_encdec"))

if unet_overall and ed_overall:
    row = {"volume_id":"Overall_Mean"}
    for k,v in unet_overall.items():
        if k!="volume_id": row[f"{k}_unet"] = v
    for k,v in ed_overall.items():
        if k!="volume_id": row[f"{k}_encdec"] = v
    cmp = pd.concat([cmp, pd.DataFrame([row])], ignore_index=True)

cmp.to_csv(OUTPUT_DIR/"compare_unet_vs_encdec.csv", index=False)
display(cmp.iloc[[-1], 9:18])
display(cmp.iloc[[-1], 1:9])


In [ ]:
# viz_compare(test_ds, encdec, model, device=DEVICE, n=6, model1_name="encdec", model2_name="UNet")
viz_compare_diagnostic(
    test_ds,
    model_1=encdec,
    model_2=model,
    device=DEVICE,
    n=6,
    samples_per_row=1,
    base_channel=1,
    show_overlay=True,
    show_boundaries=True,
    show_agreement=True,
    model_1_name="encdec",
    model_2_name="UNet"
)


In [ ]:
cleanup_cuda(model=model, optimizer=optimizer_unet, scheduler=scheduler_unet, scaler=scaler_unet)

---
## 4. Fine-tune Transfer-learning UNet encoder
### 4.1 Fine-tune Transfer-learning UNet encoder

Build the Transfer-Learning U-Net (ImageNet encoder):

In [ ]:
import os
# os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
# os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "0"
import torch.nn as nn
import torch
from huggingface_hub import snapshot_download

# print("Downloading pretrained encoder weights from Hugging Face Hub...")
# snapshot_download("smp-hub/resnet34.imagenet",
#                   allow_patterns=["*.safetensors","*.json"])

# os.environ["HF_HUB_OFFLINE"] = "1"
# print("Loading U-Net with pretrained encoder...")

import segmentation_models_pytorch as smp
# Define number of input channels (modalities) and output classes
in_channels = 4   # T1, T1Gd, T2, FLAIR
num_classes = 4   # Background + 3 tumor subregions

# Create a U-Net with an ImageNet-pretrained encoder
tl_model = smp.Unet(
    encoder_name="resnet34",           # Common choices: resnet18/34/50, efficientnet-b0/b3, etc.
    encoder_weights="imagenet",        # Use ImageNet pretrained weights
    in_channels=in_channels,           # Four MRI modalities
    classes=num_classes,               # Four segmentation classes
    activation=None,                   # Output logits (for CE + SoftDice)
).to(DEVICE)

# Freeze encoder layers during warm-up stage
def freeze_encoder(m, flag=True):
    for p in m.encoder.parameters():
        p.requires_grad = not flag

print(f"Transfer Learning Model params: {sum(p.numel() for p in tl_model.parameters())/1e6:.2f} M")
# Freeze entire encoder for warm-up
freeze_encoder(tl_model, flag=True)
print(f"Freeze fiinished.")


Define optimizers:

In [ ]:
# Two-phase training: warm-up (decoder only) -> fine-tune (full model)
LR_WARM = LR * 0.5
LR_FT = LR_WARM * 0.5
TF_EPOCHS = EPOCHS  # total TL epochs
WARMUP_EPOCHS = max(1,  int(TF_EPOCHS * 0.1))
FT_EPOCHS     = max(1, TF_EPOCHS - WARMUP_EPOCHS)
print(f"TL total epochs: {TF_EPOCHS}  (warm-up: {WARMUP_EPOCHS}, fine-tune: {FT_EPOCHS})")

Stage-wise training: freeze and unfreeze:

In [ ]:
import json

def freeze_encoder(m, flag: bool = True):
    """Freeze/unfreeze encoder params of a segmentation_models_pytorch.Unet."""
    for p in m.encoder.parameters():
        p.requires_grad = not (flag)

# --- Stage 1: warm-up decoder only (encoder frozen) --------------------------
freeze_encoder(tl_model, True)                       # freeze encoder
optim_warm = torch.optim.Adam(tl_model.parameters(), lr=LR_WARM)

# per-batch scheduler for short warm-up
sched_warm = torch.optim.lr_scheduler.OneCycleLR(
    optim_warm, max_lr=LR_WARM,
    steps_per_epoch=len(train_loader), epochs=WARMUP_EPOCHS,
    pct_start=0.1, anneal_strategy="cos"
)


es_warm = EarlyStopping(
    mode='min', patience=ES_PATIENCE, min_delta=ES_MIN_DELTA,
    ckpt_path=OUTPUT_DIR / "tl_warm_best.pt"
)

hist_tl_warm = fit(
    model=tl_model,
    train_loader=train_loader, val_loader=val_loader,
    criterion=criterion, optimizer=optim_warm, device=DEVICE,
    epochs=WARMUP_EPOCHS,
    scheduler=sched_warm, scheduler_step='batch',   # step every batch
    use_amp=(DEVICE.type == 'cuda'),
    early_stopping=es_warm,
    ckpt_path=OUTPUT_DIR / "tl_warm_best.pt",
    history_path=OUTPUT_DIR / "tl_warm_history.json",
    model_name="TL-UNet (warm)",
    num_classes=4
)

# --- Stage 2: fine-tune all layers (unfreeze encoder) ------------------------
freeze_encoder(tl_model, False)                      # unfreeze encoder

# re-create optimizer after unfreezing so it sees all params
optim_ft = torch.optim.Adam(tl_model.parameters(), lr=LR_FT)

# Can use OneCycleLR (batch) OR StepLR (epoch) OR ReduceLROnPlateau (plateau)
sched_ft = torch.optim.lr_scheduler.OneCycleLR(
    optim_ft, max_lr=LR_FT,
    steps_per_epoch=len(train_loader), epochs=FT_EPOCHS,
    pct_start=0.1, anneal_strategy="cos"
)
sched_step = 'batch'

# ReduceLROnPlateau on val_loss
# sched_ft = torch.optim.lr_scheduler.ReduceLROnPlateau(
#     optim_ft, mode='min', factor=0.5, patience=3, min_lr=1e-6
# )
# sched_step = 'plateau'

es_ft = EarlyStopping(
    mode='min', patience=ES_PATIENCE, min_delta=ES_MIN_DELTA,
    ckpt_path=OUTPUT_DIR / "tl_best.pt"
)

hist_tl_ft = fit(
    model=tl_model,
    train_loader=train_loader, val_loader=val_loader,
    criterion=criterion, optimizer=optim_ft, device=DEVICE,
    epochs=FT_EPOCHS,
    scheduler=sched_ft, scheduler_step=sched_step,
    use_amp=(DEVICE.type == 'cuda'),
    early_stopping=es_ft,
    ckpt_path=OUTPUT_DIR / "tl_best.pt",
    history_path=OUTPUT_DIR / "tl_ft_history.json",
    model_name="TL-UNet (finetune)",
    num_classes=4
)

# --- merge histories for plotting ---------------------------------
import json
hist_tl = {
    "epoch":              hist_tl_warm["epoch"] + [e + WARMUP_EPOCHS for e in hist_tl_ft["epoch"]],
    "train_loss":         hist_tl_warm["train_loss"] + hist_tl_ft["train_loss"],
    "val_loss":           hist_tl_warm["val_loss"] + hist_tl_ft["val_loss"],
    "val_dice":           hist_tl_warm["val_dice"] + hist_tl_ft["val_dice"],
}
with open(OUTPUT_DIR / "tl_training_history_merged.json", "w") as f:
    json.dump(hist_tl, f)
print("TL-UNet done. Best weights at:", OUTPUT_DIR / "tl_best.pt")


In [ ]:
# # load history from file
# with open(OUTPUT_DIR/'tl_training_history_merged.json', 'r') as f:
#     history = json.load(f)

plt.figure(figsize=(6,4)); plt.plot(hist_tl['epoch'], hist_tl['train_loss'], label='train')
plt.plot(hist_tl['epoch'], hist_tl['val_loss'], label='val'); plt.xlabel('Epoch'); plt.ylabel('Loss')
plt.title('Loss Curves'); plt.legend(); plt.tight_layout(); plt.show()

plt.figure(figsize=(6,4)); plt.plot(hist_tl['epoch'], hist_tl['val_dice'], label='val_dice')
plt.xlabel('Epoch'); plt.ylabel('Dice'); plt.title('Validation Dice'); plt.legend(); plt.tight_layout(); plt.show()

In [ ]:
# tl_model.load_state_dict(torch.load(OUTPUT_DIR/'tl_best.pt', map_location=DEVICE))
tl_model.eval()
# viz_overlays_pro(val_ds, tl_model, device=DEVICE, n=6, samples_per_row=3, base_channel=1, seed=35)
viz_overlays_pro(val_ds, tl_model, device=DEVICE, n=6, samples_per_row=2, show_error=True, base_channel=1, seed=35)

In [ ]:
tl_slice, tl_vol, tl_overall = evaluate_multiclass(
    test_loader, tl_model, device=DEVICE, add_overall_mean=True, save_csv_dir=OUTPUT_DIR, model_name="tl_unet"
)

print("Overall mean row:")
print(tl_overall)          # {'dice_Necrosis':..., 'dice_Edema':..., ..., 'volume_id':'Overall_Mean'}
display(tl_vol.tail(3))  # Overall_Mean

---
### 4.2 Compare with U-Net

Quantitative evaluation and comparison:

In [ ]:
import matplotlib.pyplot as plt

with open(OUTPUT_DIR/'unet2d_training_history.json', 'r') as f:
    hist_unet = json.load(f)
# with open(OUTPUT_DIR/'tl_training_history.json', 'r') as f:
#     hist_tl = json.load(f)


plt.figure(figsize=(6,4))
plt.plot(hist_unet["epoch"], hist_unet["val_loss"], label="UNet val loss")
plt.plot(hist_tl["epoch"], hist_tl["val_loss"], label="Transfer val loss")
plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title("Validation Loss"); plt.legend(); plt.tight_layout(); plt.show()

plt.figure(figsize=(6,4))
plt.plot(hist_unet["epoch"], hist_unet["val_dice"], label="UNet val dice")
plt.plot(hist_tl["epoch"], hist_tl["val_dice"], label="Transfer val dice")
plt.xlabel("Epoch"); plt.ylabel("Foreground Dice"); plt.title("Validation Dice"); plt.legend(); plt.tight_layout(); plt.show()

Qualitative visualization (UNet vs Transfer vs Ground Truth):

In [ ]:
# UNet model
model = UNet2D(in_ch=in_channels, out_ch=num_classes).to(DEVICE)
model.load_state_dict(torch.load(OUTPUT_DIR/"unet_best.pt", map_location=DEVICE))
model.eval()
unet_slice, unet_vol, unet_overall = evaluate_multiclass(test_loader, model, device=DEVICE, model_name="unet")

# # Transfer model
# tl_model = smp.Unet(
#     encoder_name="resnet34",           # Common choices: resnet18/34/50, efficientnet-b0/b3, etc.
#     encoder_weights="imagenet",        # Use ImageNet pretrained weights
#     in_channels=in_channels,           # Four MRI modalities
#     classes=num_classes,               # Four segmentation classes
#     activation=None,                   # Output logits (for CE + SoftDice)
# ).to(DEVICE)
# tl_model.load_state_dict(torch.load(OUTPUT_DIR/"tl_best.pt", map_location=DEVICE))
# tl_model.eval()
# tl_slice, tl_vol, tl_overall = evaluate_multiclass(test_loader, tl_model, device=DEVICE, model_name="tl_unet")


# Merge and compute per-volume + overall comparison
metric_cols = [c for c in tl_vol.columns if c != "volume_id"]
cmp = unet_vol.merge(tl_vol, on="volume_id", suffixes=("_unet", "_tl"))



if unet_overall and tl_overall:
    row = {"volume_id": "Overall_Mean"}
    for k, v in unet_overall.items():
        if k != "volume_id": row[f"{k}_unet"] = v
    for k, v in tl_overall.items():
        if k != "volume_id": row[f"{k}_tl"] = v
    import pandas as pd
    cmp = pd.concat([cmp, pd.DataFrame([row])], ignore_index=True)

cmp.to_csv(OUTPUT_DIR/"compare_unet_vs_TL.csv", index=False)
display(cmp.iloc[[-1], 9:18])
display(cmp.iloc[[-1], 1:9])

In [ ]:
# viz_compare(test_ds, model, tl_model, device=DEVICE, n=6, model1_name="UNet", model2_name="TL")
# viz_compare_triplet(
#     test_ds,
#     model1=model,            # dashed
#     model2=tl_model,               # dotted (or tl_model)
#     device=DEVICE,
#     n=6,
#     samples_per_row=3,
#     base_channel=1,              # e.g., T1Gd
#     title1="UNet",
#     title2="TL"
# )

viz_compare_diagnostic(
    test_ds,
    model_1=model,
    model_2=tl_model,
    device=DEVICE,
    n=6,
    samples_per_row=1,
    base_channel=1,
    show_overlay=True,
    show_boundaries=True,
    show_agreement=True,
    model_1_name="UNet2D",
    model_2_name="TL",
)


In [ ]:
cleanup_cuda(model=tl_model, optimizer=optim_ft, scheduler=sched_warm, scaler=scaler_ed)
cleanup_cuda(model=tl_model, optimizer=optim_ft, scheduler=sched_ft, scaler=scaler_ed)

---
## 5. Final Compare

In [ ]:
import pandas as pd
# read from csv
cmp_1 = pd.read_csv(OUTPUT_DIR/"compare_unet_vs_encdec.csv")

display(cmp_1.iloc[[-1], 9:18])
display(cmp_1.iloc[[-1], 1:9])

cmp_2 = pd.read_csv(OUTPUT_DIR/"compare_unet_vs_TL.csv")
display(cmp_2.iloc[[-1], 9:18])
# display(cmp_2.iloc[[-1], 1:9])